<a href="https://colab.research.google.com/github/atlas-github/abs_digital/blob/master/Extracting_text_from_images_PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extract information from a PDF document using [Tabula](https://pypi.org/project/tabula-py/)

Tabula isn't usually installed in most IDEs, so install the library using the code below.

In [ ]:
!pip install tabula-py

I'll be demonstrating how to extract information from page 4 of Maybank's Annual Report 2019, which can be found [here](https://www.maybank.com/en/investor-relations/reporting-events/reports/annual-reports.page).

In [ ]:
import tabula

# Read pdf into list of DataFrame
sample_list = tabula.read_pdf("Maybank Annual Report 2019 - Financial Statements (English).pdf", pages='4')

sample_list

[                                           Unnamed: 0  ...                Bank
 0                                                 NaN  ...           FY 31 Dec
 1                                                 NaN  ...           2018 2019
 2                     OPERATING RESULTS (RM’ million)  ...                 NaN
 3                                   Operating revenue  ...      26,681  26,906
 4         Pre-provisioning operating profit (“PPOP”)1  ...       9,491  10,283
 5                                    Operating profit  ...        8,748  8,415
 6                    Profit before taxation and zakat  ...        8,748  8,415
 7   Profit attributable to equity holders of the Bank  ...        7,308  7,279
 8   KEY STATEMENTS OF FINANCIAL POSITION DATA (RM’...  ...                 NaN
 9                                        Total assets  ...    456,613  464,360
 10                   Financial investments portfolio2  ...    121,354  126,286
 11                      Loans, advances

Now to convert the result into a table.

In [ ]:
df = sample_list[0]
df = df.drop([0])
df.columns = df.iloc[0]
df = df.drop([1])
#df[1, 0] = "Five-Year Group Financial Summary"
df

1,NaN,2015,2016,2017,2018,2019,2018 2019
2,OPERATING RESULTS (RM’ million),NaN,NaN,NaN,NaN,NaN,NaN
3,Operating revenue,"40,556","44,658","45,580","47,320","52,845","26,681 26,906"
4,Pre-provisioning operating profit (“PPOP”)1,"10,953","11,686","11,911","12,416","13,179","9,491 10,283"
5,Operating profit,"8,940","8,671","9,883","10,803","10,856","8,748 8,415"
6,Profit before taxation and zakat,"9,152","8,844","10,098","10,901","11,014","8,748 8,415"
7,Profit attributable to equity holders of the Bank,"6,836","6,743","7,521","8,113","8,198","7,308 7,279"
8,KEY STATEMENTS OF FINANCIAL POSITION DATA (RM’...,NaN,NaN,NaN,NaN,NaN,NaN
9,Total assets,"708,345","735,956","765,302","806,992","834,413","456,613 464,360"
10,Financial investments portfolio2,"122,166","130,902","154,373","177,952","192,830","121,354 126,286"
11,"Loans, advances and financing","453,493","477,775","485,584","507,084","513,420","230,367 226,589"


Now to clean the extracted data.

In [ ]:
import pandas as pd

#split the last column by the "  " delimiter
bank = df["2018 2019"].str.split(" ", 1, expand = True)

#combine the two dataframes
result = pd.concat([df, bank], axis = 1)

#rename the dataframe headers
result.columns = result.columns.fillna("Five-year Group Financial Summary")
result = result.rename(columns = {"2015": "2015_Group", "2016": "2016_Group", "2017": "2017_Group", "2018": "2018_Group", "2019": "2019_Group", 0: "2018_Bank", 1: "2019_Bank"})

#drop the extra column
result = result.drop(columns = {"2018 2019"})

#replace Nan with empty cells
result = result.fillna("")
result

,Five-year Group Financial Summary,2015_Group,2016_Group,2017_Group,2018_Group,2019_Group,2018_Bank,2019_Bank
2,OPERATING RESULTS (RM’ million),,,,,,,
3,Operating revenue,"40,556","44,658","45,580","47,320","52,845","26,681","26,906"
4,Pre-provisioning operating profit (“PPOP”)1,"10,953","11,686","11,911","12,416","13,179","9,491","10,283"
5,Operating profit,"8,940","8,671","9,883","10,803","10,856","8,748","8,415"
6,Profit before taxation and zakat,"9,152","8,844","10,098","10,901","11,014","8,748","8,415"
7,Profit attributable to equity holders of the Bank,"6,836","6,743","7,521","8,113","8,198","7,308","7,279"
8,KEY STATEMENTS OF FINANCIAL POSITION DATA (RM’...,,,,,,,
9,Total assets,"708,345","735,956","765,302","806,992","834,413","456,613","464,360"
10,Financial investments portfolio2,"122,166","130,902","154,373","177,952","192,830","121,354","126,286"
11,"Loans, advances and financing","453,493","477,775","485,584","507,084","513,420","230,367","226,589"


And if you would like to export the file.

In [ ]:
result.to_csv("result.csv")
from google.colab import files
files.download("result.csv")

#Extract information from Google Vision API's [OCR](https://cloud.google.com/vision/docs/ocr) (Optical Character Recognition).

Start by setting up definitions based on Google Vision API's [OCR](https://cloud.google.com/vision/docs/ocr). The GOOGLE_APPLICATION_CREDENTIALS file can be obtained by creating a service account key using this [method](https://cloud.google.com/docs/authentication/production).  

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="OCRproject-135c7e667aa9.json"

def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    #print('Texts:')
    
    return texts

Install the google-cloud-vision library if it has not been installed in your IDE. 

In [ ]:
!pip install google-cloud-vision

For this demonstration, I will be extracting the table listing electrical usage from [here](https://www.tnb.com.my/assets/images/bill_with_sstv2.png).

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

tnb_text = detect_text("bill_with_sstv2.png")
tnb_text

[locale: "ms"
description: "BIL ELEKTRIK ANDA\nTENAGA\nNASIONAL\nNo. Akaun : 220001234512\n: 1002000\n: RM350.00\nTERIMA KASIH\nNo. Kontrak\nKerana\nDeposit\nNo. Invois\nMembayar Dalam\nTempoh 30 Hari\n: 10001234\nAli bin Abu\n3\nTNB Careline\n1-300-88-5454\nE23A - 201 Sek 3\nWangsa Maju\n53300\nWP Kuala Lumpur\nTarikh Bil\nJumlahPerlu Dibayar RM311.90\n03 Okt 2018\nAmaun\nBayar Sebelum\nTunggakan\nCaj Semasa\nPenggenapan\nRM\n0.00\nTerima kasih\nRM 311.88\n0,02\nRM\nJumlah Bil\nRM\n311.90\n01.11.2018\nAmaun\n52685\nTarikh\n5\nBil Terdahulu\nRM\n02.08.2018\nBayaran Akhir\nRM\n526.85\n30.08.2018\nJenis Bacaan\nBacaan Sebenar\nTempoh Bil : 03.09.2018 - 03.10.2018 (31 Hari)\nTarif\nFaktor Prorata\n:A: Kediaman\n1,00000\nBlok Tarif (KWh)\n200\n100\n300\n300\nBlok Prorata (kWh)\n200\n100\n300\n130\nKadar (RM)\n0.218\nAmaun (RM)\n43.60\n33.40\n154.80\n70.98\n8\n0.334\n0.516\n0.546\nJumlah\n730\n302.78\nTidak Kena\nST\nKena\nST\nKeterangan\nJumlah\nKegunaan kWh\nKegunaan\nkWh\n600\n130\n730\n

Convert the OCR output into text, and take only the first value.

In [ ]:
tnb_raw = tnb_text[0].__str__()
tnb_raw

'locale: "ms"\ndescription: "BIL ELEKTRIK ANDA\\nTENAGA\\nNASIONAL\\nNo. Akaun : 220001234512\\n: 1002000\\n: RM350.00\\nTERIMA KASIH\\nNo. Kontrak\\nKerana\\nDeposit\\nNo. Invois\\nMembayar Dalam\\nTempoh 30 Hari\\n: 10001234\\nAli bin Abu\\n3\\nTNB Careline\\n1-300-88-5454\\nE23A - 201 Sek 3\\nWangsa Maju\\n53300\\nWP Kuala Lumpur\\nTarikh Bil\\nJumlahPerlu Dibayar RM311.90\\n03 Okt 2018\\nAmaun\\nBayar Sebelum\\nTunggakan\\nCaj Semasa\\nPenggenapan\\nRM\\n0.00\\nTerima kasih\\nRM 311.88\\n0,02\\nRM\\nJumlah Bil\\nRM\\n311.90\\n01.11.2018\\nAmaun\\n52685\\nTarikh\\n5\\nBil Terdahulu\\nRM\\n02.08.2018\\nBayaran Akhir\\nRM\\n526.85\\n30.08.2018\\nJenis Bacaan\\nBacaan Sebenar\\nTempoh Bil : 03.09.2018 - 03.10.2018 (31 Hari)\\nTarif\\nFaktor Prorata\\n:A: Kediaman\\n1,00000\\nBlok Tarif (KWh)\\n200\\n100\\n300\\n300\\nBlok Prorata (kWh)\\n200\\n100\\n300\\n130\\nKadar (RM)\\n0.218\\nAmaun (RM)\\n43.60\\n33.40\\n154.80\\n70.98\\n8\\n0.334\\n0.516\\n0.546\\nJumlah\\n730\\n302.78\\nTidak K

Seperate each value by the delimiter.

In [ ]:
tnb_list = list(tnb_raw.split("\\n"))
tnb_list

['locale: "ms"\ndescription: "BIL ELEKTRIK ANDA',
 'TENAGA',
 'NASIONAL',
 'No. Akaun : 220001234512',
 ': 1002000',
 ': RM350.00',
 'TERIMA KASIH',
 'No. Kontrak',
 'Kerana',
 'Deposit',
 'No. Invois',
 'Membayar Dalam',
 'Tempoh 30 Hari',
 ': 10001234',
 'Ali bin Abu',
 '3',
 'TNB Careline',
 '1-300-88-5454',
 'E23A - 201 Sek 3',
 'Wangsa Maju',
 '53300',
 'WP Kuala Lumpur',
 'Tarikh Bil',
 'JumlahPerlu Dibayar RM311.90',
 '03 Okt 2018',
 'Amaun',
 'Bayar Sebelum',
 'Tunggakan',
 'Caj Semasa',
 'Penggenapan',
 'RM',
 '0.00',
 'Terima kasih',
 'RM 311.88',
 '0,02',
 'RM',
 'Jumlah Bil',
 'RM',
 '311.90',
 '01.11.2018',
 'Amaun',
 '52685',
 'Tarikh',
 '5',
 'Bil Terdahulu',
 'RM',
 '02.08.2018',
 'Bayaran Akhir',
 'RM',
 '526.85',
 '30.08.2018',
 'Jenis Bacaan',
 'Bacaan Sebenar',
 'Tempoh Bil : 03.09.2018 - 03.10.2018 (31 Hari)',
 'Tarif',
 'Faktor Prorata',
 ':A: Kediaman',
 '1,00000',
 'Blok Tarif (KWh)',
 '200',
 '100',
 '300',
 '300',
 'Blok Prorata (kWh)',
 '200',
 '100',
 '300',

Identify the parts which contain the data I am looking for.

In [ ]:
block = tnb_list[58:63]
block

['Blok Tarif (KWh)', '200', '100', '300', '300']

In [ ]:
prorated = tnb_list[63:68]
prorated

['Blok Prorata (kWh)', '200', '100', '300', '130']

In [ ]:
rate = tnb_list[68:70] + tnb_list[76:79]
rate

['Kadar (RM)', '0.218', '0.334', '0.516', '0.546']

In [ ]:
amount = tnb_list[70:75]
amount

['Amaun (RM)', '43.60', '33.40', '154.80', '70.98']

Turn the lists into a table, and clean the table into an appropriate format.

In [ ]:
import pandas as pd
compiled = pd.DataFrame([block, prorated, rate, amount]).T
compiled

,0,1,2,3
0,Blok Tarif (KWh),Blok Prorata (kWh),Kadar (RM),Amaun (RM)
1,200,200,0.218,43.60
2,100,100,0.334,33.40
3,300,300,0.516,154.80
4,300,130,0.546,70.98


In [ ]:
compiled.columns = compiled.iloc[0]

And here's the final result. 

In [ ]:
compiled = compiled.drop([0])
compiled

,Blok Tarif (KWh),Blok Prorata (kWh),Kadar (RM),Amaun (RM)
1,200,200,0.218,43.60
2,100,100,0.334,33.40
3,300,300,0.516,154.80
4,300,130,0.546,70.98
